### Introdução
Delta Lake é uma camada de armazenamento aberta, que proporciona ACID transactions para Apache Spark e big data workloads. Esse notebook foca em demonstrar como o Delta Lake pode ser usado para permitir a evolução de esquemas em conjuntos de dados, uma capacidade vital para projetos de dados que podem crescer e mudar com o tempo. Através de exemplos práticos, veremos como adicionar colunas a DataFrames existentes, como gerenciar versões de esquemas, e como acessar dados de versões anteriores.

### Importação de Bibliotecas

In [ ]:
from delta.tables import *

Importamos as bibliotecas necessárias para trabalhar com tabelas Delta, que suportam operações ACID e permitem a evolução do esquema de DataFrames.

### Criação e Visualização de DataFrame Inicial

In [ ]:
# Caminho para o diretório Delta Lake onde os dados serão armazenados
delta_path = "/path/evolutivo"

# Criação de um DataFrame de exemplo com um esquema inicial
df = spark.createDataFrame([(1, "João", 25), (2, "Maria", 30)], ["id", "nome", "idade"])
df.show()

120 280 452 72

Criamos um DataFrame inicial com um esquema básico contendo id, nome e idade. Utilizamos show() para visualizar o DataFrame e verificar a estrutura dos dados.

### Salvando o DataFrame no Formato Delta Lake

In [ ]:
# Salvando o DataFrame no formato Delta Lake
df.write.format("delta").mode("overwrite").save(delta_path)

### Atualização do Esquema com Nova Coluna

In [ ]:
# Realizando uma atualização no esquema: adicionando uma nova coluna "cidade"
df_novo_esquema = spark.createDataFrame([(1, "João", 25, "São Paulo"), (2, "Maria", 30, "Rio de Janeiro")],
                                        ["id", "nome", "idade", "cidade"])

# Salvando o DataFrame com o novo esquema no mesmo diretório Delta Lake
df_novo_esquema.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(delta_path)

Atualizamos o esquema do DataFrame ao adicionar uma nova coluna cidade. Ao salvar o DataFrame atualizado, usamos a opção mergeSchema para mesclar o novo esquema com o esquema existente no Delta Lake.

### Visualização de Histórico de Versões

In [ ]:
# Carregando o DeltaTable a partir do caminho especificado
delta_table = DeltaTable.forPath(spark, delta_path)

# Visualizando as versões disponíveis
delta_table.history().show(truncate=False)

Carregamos a tabela Delta e visualizamos o histórico de operações, incluindo as mudanças no esquema. Isso é útil para rastrear a evolução dos dados e entender as alterações realizadas ao longo do tempo.

### Carregamento e Visualização de Dados com Esquema Atual

In [ ]:
# Lendo o DataFrame com o esquema evolutivo
df_esquema_atual = spark.read.format("delta").load(delta_path)
# Exibindo o DataFrame resultante
df_esquema_atual.show()

Carregamos o DataFrame com o esquema evolutivo atual e exibimos os dados para confirmar que a nova coluna foi adicionada corretamente.

In [ ]:
# Carregando o DataFrame de uma versão anterior do Delta Lake
df_versao_0 = spark.read.format("delta").option("versionAsOf", 0).load(delta_path)
# Exibindo o DataFrame da versão 0
df_versao_0.show()

### Limpeza de Dados

In [ ]:
dbutils.fs.rm("/path/evolutivo", recurse=True)